In [ ]:
import speech_recognition as sr
import chess
import chess.engine
import time
import pyttsx3
import chess.svg

In [ ]:
board = chess.Board()
r = sr.Recognizer()
mic = sr.Microphone()
tts = pyttsx3.init()
tts.setProperty('rate', 150)
engine = chess.engine.SimpleEngine.popen_uci(r"/Users/ayushsriv/Desktop/stockfish/12/bin/stockfish")


In [ ]:
def recognize_speech_from_mic(recognizer, microphone):
    """Transcribe speech from recorded from `microphone`.

    Returns a dictionary with three keys:
    "success": a boolean indicating whether or not the API request was
               successful
    "error":   `None` if no error occured, otherwise a string containing
               an error message if the API could not be reached or
               speech was unrecognizable
    "transcription": `None` if speech could not be transcribed,
               otherwise a string containing the transcribed text
    """
    # check that recognizer and microphone arguments are appropriate type
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    # adjust the recognizer sensitivity to ambient noise and record audio
    # from the microphone
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    # set up the response object
    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    # try recognizing the speech in the recording
    # if a RequestError or UnknownValueError exception is caught,
    #     update the response object accordingly
    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        # API was unreachable or unresponsive
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        # speech was unintelligible
        response["error"] = "Unable to recognize speech"

    return response

In [ ]:
def spoken_to_algebraic(text):
    if text == 'kingside castle':
        return 'O-O'
    elif text == 'queenside castle':
        return 'O-O-O'
    # Define a dictionary that maps chess pieces to their spoken format
    piece_names = {'night': 'N', 'knight': 'N', 'bishop': 'B', 'rook': 'R', 'queen': 'Q', 'king': 'K'}
    action_words = {'takes': 'x', 'to': '', '2':''}
    algebraic = ''
    for t in text.split(" "):
        if t in piece_names:
            algebraic+=piece_names[t]
        elif t in action_words:
            algebraic+=action_words[t]
        else:
            algebraic+=t
        
    return algebraic

In [ ]:
# Define a function that takes a chess move in algebraic notation and returns it in spoken format
def algebraic_to_spoken(move):    
    # Define a dictionary that maps chess pieces to their spoken format
    piece_names = {'P': 'pawn', 'N': 'knight', 'B': 'bishop', 'R': 'rook', 'Q': 'queen', 'K': 'king'}
    
    if move=='O-O':
        return "king side castle"
    elif move=='O-O-O':
        return "queen side castle" 
        
    # Construct the spoken format of the move
    spoken = ''
    for c in move:
        if c.isupper():
            spoken+=piece_names[c]
            spoken+=" "
        elif c=='x':
            spoken+="takes"
            spoken+=" "
        elif c=='+':
            spoken+="check"
        elif c=='#':
            spoken+="checkmate"        
        else:
            spoken+=c
    
    return spoken

In [ ]:

while not board.is_game_over():
    
    result = engine.play(board, chess.engine.Limit(time=0.1))
    
    # convert from uci to algebraic notation
    ai_move = chess.Move.from_uci(str(result.move))
    
    # text to speech
    tts.say(algebraic_to_spoken(board.san(ai_move)))
    tts.runAndWait()
    board.push(result.move)
    while True and not board.is_game_over():
        try: 
            print("Speak now")
            move = recognize_speech_from_mic(r,mic)
            move = move["transcription"].lower()
            print(move)
            move = spoken_to_algebraic(move)
            board.parse_san(move)
        except Exception as e:
            print(e)
            print("Not a valid move")
            continue
        else:
            break
    board.push_san(move)
tts.quit()

keyword Move? optional for people who want to talk while playing 
Show board
take back 



